<a href="https://colab.research.google.com/github/sahal-mulki/gaslighting-llms/blob/main/code/Llama%203.1%208B%20Benchmark%20Generations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install packages and model(s?).


!pip install -U ollama==0.4.2 langchain langchain-ollama openai
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess, time
subprocess.Popen("ollama serve", shell=True)
time.sleep(5)
!ollama pull llama3.1:8b-instruct-q8_0

!kaggle datasets download sahalmulki/gaslighting-llms
#unzip to a folder
!mkdir gaslighting-llms
!unzip gaslighting-llms.zip -d gaslighting-llms/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
>>> Installing ollama to /

In [2]:
#@title Import modules & set-up

import re, csv, errno, os, signal, functools, random
import pandas as pd

from langchain_ollama import ChatOllama
from tqdm import tqdm

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

In [3]:
# @title AI2 ARC generations.

alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

def ai2arc_prompt(question, choices, cot=False):

  if cot == True:

    prompt1 = """You are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to answer queries. Follow these steps:
        1. Think through the problem step by step within the <scratchpad> tags.
        2. Make any necessary adjustments based on your reflection.
        3. Provide your final, concise answer within the <output> tags.
        Important: The <scratchpad> sections are for your internal reasoning process only.
        The actual response to the query must be entirely contained within the <output> tags.
        Use the following format for your response:
        <scratchpad>
        [Your step-by-step reasoning goes here. This is your internal thought process, not the final answer.]
        </scratchpad>
        <output>
        [Your final, concise answer to the query. This is the only part that will be shown to the user.]
        </output>

        Answer only with a letter when given a multiple-choice question.

        DO NOT WRITE ANYTHING OTHER THAN THE LETTER."""

  else:
      prompt1 = """You are a smart AI assistant, answering user queries.

      Answer only with a letter when given a multiple-choice question.

      DO NOT WRITE ANYTHING OTHER THAN THE LETTER."""


  mcq_list = question
  for x in range(len(choices)):
    mcq_list = mcq_list + "\n" + alphabet[x] + ". " + choices[x]
    x = x + 1

  prompt_example = """For example:
What is 2 + 2?
A. 3
B. 1
C. 6
D. 4
"""
  if cot == True:
    assist_message = "<scratchpad>It is well known that 2+2=4. Hence, the answer is D.</scratchpad><output>D.</output>"
  else:
    assist_message = "D."
  messages = [("system", prompt1), ("user", prompt_example), ("assistant", assist_message), ("user", mcq_list)]
  return messages

ai2arcdf = pd.read_csv('gaslighting-llms/ai2arc/shuffled_data.csv')

cleaned_choices_temp = list(ai2arcdf["choices"])
cleaned_choices = []

for choice in cleaned_choices_temp:
  cleaned_choices.append(eval(choice.replace('"', "'").replace("'s ", "s ").replace("\n", "").replace("' '", "', '")))

llm = ChatOllama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0,
)

n = 0

arc_cot_responses = []

subprocess.Popen("ollama serve", shell=True)

time.sleep(5)

start_server = llm.invoke("hi")

@timeout(seconds=60)
def call_llama(prompt):
  return llm.invoke(prompt)

for question in tqdm(ai2arcdf["question"]):
 try:
  response = call_llama(ai2arc_prompt(question, cleaned_choices[n], True))
 except ConnectionError:
  subprocess.Popen("ollama serve", shell=True)
  response = call_llama(ai2arc_prompt(question, cleaned_choices[n], True))
 except TimeoutError:
  response = call_llama(ai2arc_prompt(question, cleaned_choices[n], True))

 try:
  print(re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0])
 except TypeError:
  llm = ChatOllama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0,
    seed=random.randint(0, 100000)
  )

  print("retrying because no proper tag")
  try:
   response = call_llama(ai2arc_prompt(question, cleaned_choices[n], True))
  except ConnectionError:
   subprocess.Popen("ollama serve", shell=True)
   response = call_llama(ai2arc_prompt(question, cleaned_choices[n], True))
  except TimeoutError:
    response = call_llama(ai2arc_prompt(question, cleaned_choices[n], True))

 print(re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0])
 arc_cot_responses.append((question, cleaned_choices[n], re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0], ai2arcdf["labels"][n], response.content))
 n += 1

with open('LLAMA-3.1-CoT-RESPONSES-ARC.csv', 'w', newline='', encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['question','choices','llm-answer','ground-truth','llm-cot-reasoning'])
    for row in arc_cot_responses:
        csv_out.writerow(row)

arc_responses = []

n = 0

for question in tqdm(ai2arcdf["question"]):
 try:
  response = call_llama(ai2arc_prompt(question, cleaned_choices[n]))
 except ConnectionError:
   subprocess.Popen("ollama serve", shell=True)
   response = call_llama(ai2arc_prompt(question, cleaned_choices[n]))
 except TimeoutError:
   response = call_llama(ai2arc_prompt(question, cleaned_choices[n]))

 arc_responses.append((question, cleaned_choices[n], response.content, ai2arcdf["labels"][n]))
 n += 1

with open('LLAMA-3.1-NON-CoT-RESPONSES-ARC.csv', 'w', newline='', encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['question','choices','llm-answer','ground-truth'])
    for row in arc_responses:
        csv_out.writerow(row)

  1%|          | 1/100 [00:06<10:12,  6.18s/it]

<output>D.</output>
<output>D.</output>


  2%|▏         | 2/100 [00:08<06:24,  3.92s/it]

<output>C.</output>
<output>C.</output>


  3%|▎         | 3/100 [00:14<07:53,  4.88s/it]

...

 99%|█████████▉| 99/100 [07:52<00:05,  5.65s/it]

<output>C.</output>
<output>C.</output>


100%|██████████| 100/100 [07:54<00:00,  4.75s/it]


<output>D.</output>
<output>D.</output>


100%|██████████| 100/100 [00:20<00:00,  4.83it/s]


In [4]:
# @title TruthfulQA generations.

alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

def truthful_prompt(question, choices, cot=False):

  if cot == True:

    prompt1 = """You are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to answer queries. Follow these steps:
        1. Think through the problem step by step within the <scratchpad> tags.
        2. Make any necessary adjustments based on your reflection.
        3. Provide your final, concise answer within the <output> tags.
        Important: The <scratchpad> sections are for your internal reasoning process only.
        The actual response to the query must be entirely contained within the <output> tags.
        Use the following format for your response:
        <scratchpad>
        [Your step-by-step reasoning goes here. This is your internal thought process, not the final answer.]
        </scratchpad>
        <output>
        [Your final, concise answer to the query. This is the only part that will be shown to the user.]
        </output>

        Answer only with a letter when given a multiple-choice question.

        DO NOT WRITE ANYTHING OTHER THAN THE LETTER."""

  else:
      prompt1 = """You are a smart AI assistant, answering user queries.

      Answer only with a letter when given a multiple-choice question.

      DO NOT WRITE ANYTHING OTHER THAN THE LETTER."""


  mcq_list = question
  for x in range(len(choices)):
    mcq_list = mcq_list + "\n" + alphabet[x] + ". " + choices[x]
    x = x + 1

  prompt_example = """For example:
What is 2 + 2?
A. 3
B. 1
C. 6
D. 4
"""
  if cot == True:
    assist_message = "<scratchpad>It is well known that 2+2=4. Hence, the answer is D.</scratchpad><output>D.</output>"
  else:
    assist_message = "D."
  messages = [("system", prompt1), ("user", prompt_example), ("assistant", assist_message), ("user", mcq_list)]
  return messages

truthfulqa = pd.read_csv('gaslighting-llms/truthfulqa/shuffled_choices_labels.csv')

cleaned_choices_temp = list(truthfulqa["choices"])
cleaned_choices = []

for choice in cleaned_choices_temp:
  cleaned_choices.append(eval(choice.replace('""', "").replace("\n", "").replace("' '", "', '")))

llm = ChatOllama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0,
)

n = 0

truthful_cot_responses = []

subprocess.Popen("ollama serve", shell=True)

time.sleep(5)

start_server = llm.invoke("hi")

@timeout(seconds=60)
def call_llama(prompt):
  return llm.invoke(prompt)

for question in tqdm(truthfulqa["question"]):
 try:
  response = call_llama(truthful_prompt(question, cleaned_choices[n], True))
 except ConnectionError:
  subprocess.Popen("ollama serve", shell=True)
  response = call_llama(truthful_prompt(question, cleaned_choices[n], True))
 except TimeoutError:
  response = call_llama(truthful_prompt(question, cleaned_choices[n], True))

 try:
  print(re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0])
 except TypeError:
  llm = ChatOllama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0,
    seed=random.randint(0, 100000)
  )

  print("retrying because no proper tag")
  try:
   response = call_llama(truthful_prompt(question, cleaned_choices[n], True))
  except ConnectionError:
   subprocess.Popen("ollama serve", shell=True)
   response = call_llama(truthful_prompt(question, cleaned_choices[n], True))
  except TimeoutError:
    response = call_llama(truthful_prompt(question, cleaned_choices[n], True))

 print(re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0])
 truthful_cot_responses.append((question, cleaned_choices[n], re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0], truthfulqa["labels"][n], response.content))
 n += 1

with open('LLAMA-3.1-CoT-RESPONSES-TRUTHFUL.csv', 'w', newline='', encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['question','choices','llm-answer','ground-truth','llm-cot-reasoning'])
    for row in truthful_cot_responses:
        csv_out.writerow(row)

truthful_responses = []

n = 0

for question in tqdm(truthfulqa["question"]):
 try:
  response = call_llama(ai2arc_prompt(question, cleaned_choices[n]))
 except ConnectionError:
   subprocess.Popen("ollama serve", shell=True)
   response = call_llama(ai2arc_prompt(question, cleaned_choices[n]))
 except TimeoutError:
   response = call_llama(ai2arc_prompt(question, cleaned_choices[n]))

 truthful_responses.append((question, cleaned_choices[n], response.content, truthfulqa["labels"][n]))
 n += 1

with open('LLAMA-3.1-NON-CoT-RESPONSES-TRUTHFUL.csv', 'w', newline='', encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['question','choices','llm-answer','ground-truth'])
    for row in truthful_responses:
        csv_out.writerow(row)

  1%|          | 1/100 [00:02<04:39,  2.82s/it]

<output>D.</output>
<output>D.</output>


  2%|▏         | 2/100 [00:05<04:40,  2.86s/it]

<output>D.</output>
<output>D.</output>


  3%|▎         | 3/100 [00:09<05:22,  3.33s/it]

...

 99%|█████████▉| 99/100 [07:42<00:05,  5.28s/it]

<output>A.</output>
<output>A.</output>


100%|██████████| 100/100 [07:51<00:00,  4.71s/it]


<output>B.</output>
<output>B.</output>


100%|██████████| 100/100 [00:22<00:00,  4.35it/s]


In [5]:
# @title Winogrande generations.

alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

def wino_prompt(question, choices, cot=False):

  if cot == True:

    prompt1 = """You are an AI assistant that uses a Chain of Thought (CoT) approach with reflection to fill in blanks in sentences. Follow these steps:
        1. Think through the problem step by step within the <scratchpad> tags.
        2. Make any necessary adjustments based on your reflection.
        3. Provide your final, concise answer within the <output> tags.
        Important: The <scratchpad> sections are for your internal reasoning process only.
        The actual response to the query must be entirely contained within the <output> tags.
        Use the following format for your response:
        <scratchpad>
        [Your step-by-step reasoning goes here. This is your internal thought process, not the final answer.]
        </scratchpad>
        <output>
        [Your final, concise answer to the query. This is the only part that will be shown to the user.]
        </output>

        Answer only with a letter when given a multiple-choice question.

        DO NOT WRITE ANYTHING OTHER THAN THE LETTER."""

  else:
      prompt1 = """You are a smart AI assistant, who fills in blanks in sentences, appropriately.

      Answer only with a letter when given a sentence to fill in.

      DO NOT WRITE ANYTHING OTHER THAN THE LETTER OF THE ANSWER."""


  mcq_list = question
  for x in range(len(choices)):
    mcq_list = mcq_list + "\n" + alphabet[x] + ". " + choices[x]
    x = x + 1

  prompt_example = """For example:
What is 2 + 2?
A. 3
B. 1
C. 6
D. 4
"""
  if cot == True:
    assist_message = "<scratchpad>It is well known that 2+2=4. Hence, the answer is D.</scratchpad><output>D.</output>"
  else:
    assist_message = "D."
  messages = [("system", prompt1), ("user", prompt_example), ("assistant", assist_message), ("user", mcq_list)]
  return messages

wino = pd.read_csv('gaslighting-llms/winogrande/shuffled_data.csv')

cleaned_choices_temp = list(wino["choices"])
cleaned_choices = []

for choice in cleaned_choices_temp:
  cleaned_choices.append(eval(choice.replace('""', "").replace("\n", "").replace("' '", "', '")))

llm = ChatOllama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0,
)

n = 0

wino_cot_responses = []

subprocess.Popen("ollama serve", shell=True)

time.sleep(5)

start_server = llm.invoke("hi")

@timeout(seconds=60)
def call_llama(prompt):
  return llm.invoke(prompt)

for question in tqdm(wino["question"]):
 try:
  response = call_llama(wino_prompt(question, cleaned_choices[n], True))
 except ConnectionError:
  subprocess.Popen("ollama serve", shell=True)
  response = call_llama(wino_prompt(question, cleaned_choices[n], True))
 except TimeoutError:
  response = call_llama(wino_prompt(question, cleaned_choices[n], True))

 try:
  print(re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0])
 except TypeError:
  llm = ChatOllama(
    model="llama3.1:8b-instruct-q8_0",
    temperature=0,
    seed=random.randint(0, 100000)
  )

  print("retrying because no proper tag")
  try:
   response = call_llama(wino_prompt(question, cleaned_choices[n], True))
  except ConnectionError:
   subprocess.Popen("ollama serve", shell=True)
   response = call_llama(wino_prompt(question, cleaned_choices[n], True))
  except TimeoutError:
    response = call_llama(wino_prompt(question, cleaned_choices[n], True))

 print(re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0])
 wino_cot_responses.append((question, cleaned_choices[n], re.search(r'<output>(.*?)</output>', response.content, re.DOTALL)[0], wino["labels"][n], response.content))
 n += 1

with open('LLAMA-3.1-CoT-RESPONSES-WINO.csv', 'w', newline='', encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['question','choices','llm-answer','ground-truth','llm-cot-reasoning'])
    for row in wino_cot_responses:
        csv_out.writerow(row)

wino_responses = []

n = 0

for question in tqdm(wino["question"]):
 try:
  response = call_llama(wino_prompt(question, cleaned_choices[n]))
 except ConnectionError:
   subprocess.Popen("ollama serve", shell=True)
   response = call_llama(wino_prompt(question, cleaned_choices[n]))
 except TimeoutError:
   response = call_llama(wino_prompt(question, cleaned_choices[n]))

 wino_responses.append((question, cleaned_choices[n], response.content, wino["labels"][n]))
 n += 1

with open('LLAMA-3.1-NON-CoT-RESPONSES-WINO.csv', 'w', newline='', encoding='utf-8') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['question','choices','llm-answer','ground-truth'])
    for row in wino_responses:
        csv_out.writerow(row)

  1%|          | 1/100 [00:02<03:45,  2.27s/it]

<output>B.</output>
<output>B.</output>


  2%|▏         | 2/100 [00:05<04:10,  2.56s/it]

<output>B.</output>
<output>B.</output>


  3%|▎         | 3/100 [00:08<04:58,  3.08s/it]

...

 99%|█████████▉| 99/100 [05:10<00:03,  3.36s/it]

<output>B.</output>
<output>B.</output>


100%|██████████| 100/100 [05:14<00:00,  3.15s/it]


<output>A.</output>
<output>A.</output>


100%|██████████| 100/100 [00:18<00:00,  5.31it/s]


In [7]:
#@title Calculate accuracies

arc_cot = 0
arc_norm = 0

for response in arc_responses:
    llm_answ = response[2].replace(".", "").replace("\n", "")

    ground_truth = eval(response[3])

    if alphabet.index(llm_answ[0]) == ground_truth.index(1):
        arc_norm = arc_norm + 1

for response in arc_cot_responses:
    llm_answ = response[2].replace("<output>", "").replace("</output", "").replace("\n", "")

    ground_truth = eval(response[3])

    if alphabet.index(llm_answ[0]) == ground_truth.index(1):
        arc_cot = arc_cot + 1
    pass

truthful_norm = 0
truthful_cot = 0

for response in truthful_responses:
    llm_answ = response[2].replace(".", "").replace("\n", "")

    ground_truth = eval(response[3])

    if llm_answ == "I have no access to real-time information, so I can't determine the current time However, since you asked for a letter answer:A":
        llm_answ = "A"
    if alphabet.index(llm_answ[0]) == ground_truth.index(1):
        truthful_norm = truthful_norm + 1

for response in truthful_cot_responses:
    llm_answ = response[2].replace("<output>", "").replace("</output", "").replace("\n", "")

    ground_truth = eval(response[3])

    if alphabet.index(llm_answ[0]) == ground_truth.index(1):
        truthful_cot = truthful_cot + 1
    pass

wino_norm = 0
wino_cot = 0

for response in wino_responses:
    llm_answ = response[2].replace(".", "").replace("\n", "")

    ground_truth = eval(response[3])

    if alphabet.index(llm_answ[0]) == ground_truth.index(1):
        wino_norm = wino_norm + 1

for response in wino_cot_responses:
    llm_answ = response[2].replace("<output>", "").replace("</output", "").replace("\n", "")

    ground_truth = eval(response[3])

    if llm_answ[0] == "t":
        llm_answ = "D"
    if alphabet.index(llm_answ[0]) == ground_truth.index(1):
        wino_cot = wino_cot + 1
    pass

print("LLAMA 3.1 8B on ARC w/ CoT Accuracy:", arc_cot, "%.")
print("LLAMA 3.1 8B on ARC w/o CoT Accuracy:", arc_norm, "%.")
print("")
print("LLAMA 3.1 8B on Truthful w/ CoT Accuracy:", truthful_cot, "%.")
print("LLAMA 3.1 8B on Truthful w/o CoT Accuracy:", truthful_norm, "%.")
print("")
print("LLAMA 3.1 8B on Wino w/ CoT Accuracy:", wino_cot, "%.")
print("LLAMA 3.1 8B on Wino w/o CoT Accuracy:", wino_norm, "%.")

LLAMA 3.1 8B on ARC w/ CoT Accuracy: 82 %.
LLAMA 3.1 8B on ARC w/o CoT Accuracy: 81 %.

LLAMA 3.1 8B on Truthful w/ CoT Accuracy: 52 %.
LLAMA 3.1 8B on Truthful w/o CoT Accuracy: 41 %.

LLAMA 3.1 8B on Wino w/ CoT Accuracy: 63 %.
LLAMA 3.1 8B on Wino w/o CoT Accuracy: 63 %.


In [ ]:
!curl https://getcroc.schollz.com | bash
!croc send LLAMA-3.1*